In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

os.chdir('/content/drive/MyDrive')

#1. Prepair data

In [ ]:
%cd /content/drive/MyDrive/Solo/data

/content/drive/MyDrive/Solo/data


In [ ]:
# Statistic number of image in each set
test_set = os.listdir('/content/drive/MyDrive/Solo/data/XRAYBoneDataset/test/JPEGImages')
train_set = os.listdir('/content/drive/MyDrive/Solo/data/XRAYBoneDataset/train/JPEGImages')
val_set = os.listdir('/content/drive/MyDrive/Solo/data/XRAYBoneDataset/val/JPEGImages')

print('Number of images: ')
print(f'Train set: {len(train_set)} images')
print(f'Validation set: {len(val_set)} images')
print(f'Test set: {len(test_set)} images')

Number of images: 
Train set: 3199 images
Validation set: 801 images
Test set: 801 images


#2. SOLOv2

In [ ]:
# Install pretrained weights
!wget https://cloudstor.aarnet.edu.au/plus/s/chF3VKQT4RDoEqC/download -O SOLOv2_R50_3x.pth

In [ ]:
!git clone https://github.com/aim-uofa/AdelaiDet.git

Cloning into 'AdelaiDet'...
remote: Enumerating objects: 183, done.
remote: Counting objects: 100% (183/183), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 1916 (delta 118), reused 164 (delta 111), pack-reused 1733
Receiving objects: 100% (1916/1916), 507.64 KiB | 2.21 MiB/s, done.
Resolving deltas: 100% (1162/1162), done.
Checking out files: 100% (205/205), done.


In [ ]:
%cd /content/drive/MyDrive/Solo/AdelaiDet

/content/drive/MyDrive/Solo/AdelaiDet


In [ ]:
# install dependencies: 
!pip install pyyaml==5.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

In [ ]:
# install detectron2: (Colab has CUDA 10.1 + torch 1.8)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
import torch
assert torch.__version__.startswith("1.8")   # need to manually install torch 1.8 if Colab changes its default version
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
# exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

In [ ]:
# Build adet package
!python setup.py build develop

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
#visualize training data

my_dataset_train_metadata = MetadataCatalog.get("train_data")
dataset_dicts = DatasetCatalog.get("train_data")

import random
import matplotlib.pyplot as plt
from detectron2.utils.visualizer import Visualizer

for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d['file_name'])
    visualizer = Visualizer(img[:, :, ::-1], metadata=my_dataset_train_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])


## Training and evaluation


In [ ]:
# We make our own config to custom model for training in our dataset
# Our xray dataset has 3600 images for train, 800 images for validation and 800 images for test.
# We achieve our best model after running 600 iterations with some notifications: lr = 0.0025 (decay=True), optimizer=SGD, batch_size=8
!OMP_NUM_THREADS=1 python tools/train_net.py \
    --config-file configs/SOLOv2/R50_3x.yaml \
    --num-gpus 1 

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 

Command Line Args: Namespace(config_file='configs/SOLOv2/R50_3x.yaml', dist_url='tcp://127.0.0.1:49152', eval_only=False, machine_rank=0, num_gpus=1, num_machines=1, opts=[], resume=False)
[04/08 08:17:59 detectron2]: Rank of current process: 0. World size: 1
[04/08 08:18:00 detectron2]: Environment info:
----------------------  ---------------------------------------------------------------
sys.platform            linux
Python                  3.7.10 (default, Feb 20 2021, 21:17:23) [GCC 7.5.0]
numpy                   1.19.5
detectron2              0.4 @/usr/local/lib/python3.7/dist-packages/detectron2
Compiler                GCC 7.3
CUDA compiler           CUDA 10.1
detectron2 arch flags   3.7, 5.0, 5.2, 6.0, 6.1, 7.0, 7.5
DETECTRON2_ENV_MODULE   <not set>
PyTorch                 1.8.1+cu101 @/usr/local/lib/python3.7/dist-packag

## Test

In [36]:

!python demo/demo.py \
    --config-file configs/SOLOv2/R50_3x.yaml \
    --input /content/drive/MyDrive/Solo/AdelaiDet/datasets/xray/test/JPEGImages/patient09379_study1_negative_image1.jpg \
    --output output.jpg \
    --opts MODEL.WEIGHTS /content/drive/MyDrive/Solo/AdelaiDet/training_dir/SOLOv2_R50_3x/model_final.pth

[04/08 08:44:40 detectron2]: Arguments: Namespace(confidence_threshold=0.3, config_file='configs/SOLOv2/R50_3x.yaml', input=['/content/drive/MyDrive/Solo/AdelaiDet/datasets/xray/test/JPEGImages/patient09379_study1_negative_image1.jpg'], opts=['MODEL.WEIGHTS', '/content/drive/MyDrive/Solo/AdelaiDet/training_dir/SOLOv2_R50_3x/model_final.pth'], output='output.jpg', video_input=None, webcam=False)
  0% 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3503: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now u

#3. SOLOv1 (Not complete because this code compatible in conda environment)

In [ ]:
!git clone https://github.com/WXinlong/SOLO.git


Cloning into 'SOLO'...
remote: Enumerating objects: 6428, done.
remote: Total 6428 (delta 0), reused 0 (delta 0), pack-reused 6428
Receiving objects: 100% (6428/6428), 6.05 MiB | 6.97 MiB/s, done.
Resolving deltas: 100% (4417/4417), done.
Checking out files: 100% (457/457), done.


In [ ]:
cd SOLO

/content/drive/MyDrive/SOLO


In [ ]:
ls

configs/  docs/           mmdet/      requirements/     tests/
demo/     highlights.png  pytest.ini  requirements.txt  tools/
docker/   LICENSE         README.md   setup.py


In [ ]:
!pip install -r requirements/build.txt
!pip install "git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI"
!pip install -v -e .  # or "python setup.py develop"

In [ ]:
# Install mmcv

!pip uninstall mmcv mmcv-full
# !git clone https://github.com/open-mmlab/mmcv.git
%cd ./../mmcv
!MMCV_WITH_OPS=1 pip install -e .

Uninstalling mmcv-0.2.16:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/mmcv-0.2.16.dist-info/*
    /usr/local/lib/python3.7/dist-packages/mmcv/*
Proceed (y/n)? y
  Successfully uninstalled mmcv-0.2.16
Can't uninstall 'mmcv-full'. No files were found to uninstall.
/content/drive/My Drive/mmcv
Obtaining file:///content/drive/My%20Drive/mmcv
  Found existing installation: mmcv-full 1.3.0
    Can't uninstall 'mmcv-full'. No files were found to uninstall.
  Running setup.py develop for mmcv-full


In [ ]:
%cd /content/drive/MyDrive/SOLO

/content/drive/MyDrive/SOLO


In [ ]:
!export COCO_ROOT=/content/drive/MyDrive/SOLO/data

In [ ]:
!python tools/train.py configs/solo/solo_r50_fpn_8gpu_1x.py

2021-04-07 10:57:43,898 - mmdet - INFO - Distributed training: False
2021-04-07 10:57:43,899 - mmdet - INFO - MMDetection Version: 2.11.0
2021-04-07 10:57:43,900 - mmdet - INFO - Config:
/content/drive/My Drive/SOLO/configs/solo/solo_r50_fpn_8gpu_1x.py
# model settings
model = dict(
    type='SOLO',
    pretrained='torchvision://resnet50',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3), # C2, C3, C4, C5
        frozen_stages=1,
        style='pytorch'),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        start_level=0,
        num_outs=5),
    bbox_head=dict(
        type='SOLOHead',
        num_classes=2,
        in_channels=256,
        stacked_convs=7,
        seg_feat_channels=256,
        strides=[8, 8, 16, 32, 32],
        scale_ranges=((1, 96), (48, 192), (96, 384), (192, 768), (384, 2048)),
        sigma=0.2,
        num_grids=[40, 36, 24, 16, 12],
  